<a href="https://colab.research.google.com/github/GaelleNovales/Trendr/blob/master/Tweetsearchipynb_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ressources

##Téléchargement des ressources



In [87]:
!pip install nltk
!pip install twint
!pip install the Twython library 
!pip install emoji

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('sentiwordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

##Imports

In [0]:
import twint
from datetime import datetime
import emoji
import regex
import re
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import pandas as pd
from textblob import TextBlob
import csv
from datetime import date, datetime, timedelta
from collections import Counter
import json

#Création du projet

## Importation des tweets selon un hashtag


In [0]:
def export_tweets(mention,taille,day):
  date = datetime.now()
  date_format = str(date.year)+'-'+str(date.month)+'-'+str(date.day)
  
  date_end = date - timedelta(days=day)
  date_end_format = str(date_end.year)+'-'+str(date_end.month)+'-'+str(date_end.day)

  # Configure
  c = twint.Config()
  c.Search = mention
  #c.Store_csv = True
  #c.Output = c.Search+"_saved.csv"
  c.Limit = taille
  c.Until = date_end_format
  c.Lang = 'en'
  c.Pandas = True

  # Run
  twint.run.Search(c)

  twint_data = twint.storage.panda.Tweets_df
  return twint_data

# def export_tweets(mention,taille):
#   date = datetime.now()
#   date_format = str(date.year)+'-'+str(date.month)+'-'+str(date.day)
  
#   date_end = date - timedelta(days=7)
#   date_end_format = str(date_end.year)+'-'+str(date_end.month)+'-'+str(date_end.day)

#   # Configure
#   c = twint.Config()
#   c.Search = mention
#   c.Store_csv = True
#   c.Output = c.Search+"_saved.csv"
#   c.Limit = taille
#   c.Since = date_end_format
#   c.Until = date_format
#   c.Lang = 'en'
#   c.Pandas = True

#   # Run
#   twint.run.Search(c)

#   twint_data = twint.storage.panda.Tweets_df
#   return twint_data

##Extraction des emojis

In [0]:
def extract_emojis(a_list):
    emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
    r = re.compile('|'.join(re.escape(p) for p in emojis_list))
    aux=[' '.join(r.findall(s)) for s in a_list]
    return(aux)

##Traduire les abréviations

In [0]:
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))

##Cleaning du texte

In [0]:
def clean_text(text,mention):
    """
    This function takes as input a text on which several 
    NLTK algorithms will be applied in order to preprocess it
    """
    # Remplacer les abréviations
    #print(text)
    #print(translator(text))
    text = translator(text)

    tokens = word_tokenize(text)
      
    # Remove the punctuations
    tokens = [word for word in tokens if word.isalpha()]
    # Lower the tokens
    tokens = [word.lower() for word in tokens]

    # Remove stopword
    tokens = [word for word in tokens if not word in stopwords.words("english")]

    # Corriger les fautes
    #for word in tokens:
    #  print(word)
    #  print(str(TextBlob(word).correct()))
    tokens = [str(TextBlob(word).correct()) for word in tokens if word != mention]

    # Lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word, pos = "v") for word in tokens]
    tokens = [lemma.lemmatize(word, pos = "n") for word in tokens]
    return tokens

#Execution du programme

In [0]:
def prog(mention,taille,day):
  Tweets_df = export_tweets(mention,taille,day)

  tweet_cleaned = []
  for i in range(len(Tweets_df)):
    tweet_cleaned.append(clean_text(Tweets_df['tweet'][i],mention))  

  emojisExtracted = extract_emojis(Tweets_df['tweet'])
  Tweets_df["tweet_emojis"] = emojisExtracted
  Tweets_df["tweet_cleaned"] = tweet_cleaned
  return Tweets_df

In [205]:
#pepsi coca fanta orangina
#df.drop('column_name', axis=1, inplace=True)
frames = [prog("pepsi",150,0), prog("pepsi",150,1), prog("pepsi",150,2),prog("pepsi",150,3),prog("pepsi",150,4),prog("pepsi",150,5),prog("pepsi",150,6)]
Tweets_df_pepsi = pd.concat(frames)
Tweets_df_pepsi

frames = [prog("coca",150,0), prog("coca",150,1), prog("coca",150,2),prog("coca",150,3),prog("coca",150,4),prog("coca",150,5),prog("coca",150,6)]
Tweets_df_coca = pd.concat(frames)
Tweets_df_coca

frames = [prog("fanta",150,0), prog("fanta",150,1), prog("fanta",150,2),prog("fanta",150,3),prog("fanta",150,4),prog("fanta",150,5),prog("fanta",150,6)]
Tweets_df_fanta = pd.concat(frames)
Tweets_df_fanta

frames = [prog("orangina",150,0), prog("orangina",150,1), prog("orangina",150,2),prog("orangina",150,3),prog("orangina",150,4),prog("orangina",150,5),prog("orangina",150,6)]
Tweets_df_orangina = pd.concat(frames)
Tweets_df_orangina
#Tweets_df_coca = prog("coca",150)
#Tweets_df_fanta = prog("fanta",150)
#Tweets_df_orangina = prog("orangina",150)

1244776216537608193 2020-03-30 23:59:11 UTC <MiniMeInsights> New cold-activated colour changing Pepsi can to celebrate summer  https://www.minimeinsights.com/2020/03/31/new-cold-activated-colour-changing-pepsi-can-to-celebrate-summer/ … via @MiniMeInsights
1244775770842595329 2020-03-30 23:57:25 UTC <TheSolarGazer> Ever watched the film "The Invention of Lying"? It was brilliant. My fave was the Pepsi lorry that said "When Coke is not available".
1244775765100638213 2020-03-30 23:57:23 UTC <BeastieBoy70> You can try using crystal Pepsi as hand sanitizer.
1244775758171639808 2020-03-30 23:57:22 UTC <Keith_Ziegler67> That is so me. I absolutely hate...no despise pepsi!
1244775751997624326 2020-03-30 23:57:20 UTC <Thedoctorand> I Pepsi what you did there.
1244775608464429058 2020-03-30 23:56:46 UTC <Jonalaniz> What about SAINT PEPSI x SKYLAR SPENCE?
1244775493821497352 2020-03-30 23:56:19 UTC <trystan_raphs> my pussy tastes like pepsi cola
1244775452855734272 2020-03-30 23:56:09 UTC <Comp

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_emojis,tweet_cleaned
0,1244765267193929728,1244683315182997506,1585610140000,2020-03-30 23:15:40,UTC,,No you MY squishy,[],[],1181662130900291584,1181662130900291584,rxplinewhxre,💕✨💕,3,05,https://twitter.com/rxplinewhxre/status/124476...,False,1,0,0,,orangina,,,,,,,"[{'user_id': '1181662130900291584', 'username'...",,,,,,[squishy]
1,1244754764711301120,1244723330571091968,1585607636000,2020-03-30 22:33:56,UTC,,amazing,[],[],343035775,343035775,orangina17,Georgina Jane Petty,2,05,https://twitter.com/orangina17/status/12447547...,False,0,0,0,,orangina,,,,,,,"[{'user_id': '343035775', 'username': 'orangin...",,,,,,[amaze]
2,1244753444205928450,1244753444205928450,1585607322000,2020-03-30 22:28:42,UTC,,Well night all...,[],[],343035775,343035775,orangina17,Georgina Jane Petty,5,14,https://twitter.com/orangina17/status/12447534...,False,0,0,0,,orangina,,,,,,,"[{'user_id': '343035775', 'username': 'orangin...",,,,,,"[well, night]"
3,1244752929640394755,1244751955412561920,1585607199000,2020-03-30 22:26:39,UTC,,"Honestly....🤦‍♀️(them, not you) pic.twitter.co...",[],[],343035775,343035775,orangina17,Georgina Jane Petty,4,04,https://twitter.com/orangina17/status/12447529...,False,1,0,0,,orangina,,,,,,,"[{'user_id': '343035775', 'username': 'orangin...",,,,,🤦 ♀,[honestly]
4,1244752821431545857,1244752578220634112,1585607173000,2020-03-30 22:26:13,UTC,,Thanks you too,[],[],343035775,343035775,orangina17,Georgina Jane Petty,3,20,https://twitter.com/orangina17/status/12447528...,False,1,1,0,,orangina,,,,,,,"[{'user_id': '343035775', 'username': 'orangin...",,,,,,[thank]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1241380440755310592,1240475914246467589,1584803135000,2020-03-21 15:05:35,UTC,,I can’t look or listen to ORANGINA anymore! Wh...,[],[],2160190735,2160190735,lisamayo19,lisa mayo,7,21,https://twitter.com/lisamayo19/status/12413804...,False,0,0,0,,orangina,,,,,,,"[{'user_id': '2160190735', 'username': 'lisama...",,,,,,"[look, listen, anymore, still, to, game, show,..."
156,1241355021557665795,1241355021557665795,1584797075000,2020-03-21 13:24:35,UTC,,This visual metaphor is for @Orangina on http...,[#vismetrgb],[],2428936177,2428936177,MetaphorMagnet,MetaphorIsMyBusiness,7,17,https://twitter.com/MetaphorMagnet/status/1241...,False,0,0,0,,orangina,,,,,,,"[{'user_id': '2428936177', 'username': 'Metaph...",,,,,,"[visual, metaphor, http, thank, colour, title,..."
157,1241352608037498880,1241352608037498880,1584796499000,2020-03-21 13:14:59,UTC,,Orangina has ensured a bloodbath. He assumed h...,[],[],1168504033029447686,1168504033029447686,some_kids_mom,Danielle isolated before it was cool,1,01,https://twitter.com/some_kids_mom/status/12413...,False,2,1,0,https://twitter.com/JustinWolfers/status/12411...,orangina,,,,,,,"[{'user_id': '1168504033029447686', 'username'...",,,,,,"[ensue, bloodbath, assume, privilege, way, one..."
158,1241314037583011841,1241314037583011841,1584787303000,2020-03-21 10:41:43,UTC,,"okay i did it, for my first attempt it's not s...",[],[],1238482638303789056,1238482638303789056,orangina_rose,panda girl (check 📌),6,15,https://twitter.com/orangina_rose/status/12413...,False,1,0,0,,orangina,,,,,,,"[{'user_id': '1238482638303789056', 'username'...",,,,,,"[okay, first, attempt, bad]"


## On assemble les 4 recherches (pepsi, coca, fanta, orangina)

In [206]:
frames = [Tweets_df_pepsi, Tweets_df_coca, Tweets_df_fanta, Tweets_df_orangina]
result = pd.concat(frames)
result.head(25)
result.to_csv("./test_final.csv")

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_emojis,tweet_cleaned
0,1244776216537608193,1244776216537608193,1585612751000,2020-03-30 23:59:11,UTC,,New cold-activated colour changing Pepsi can t...,[],[],738937411963490304,738937411963490304,MiniMeInsights,Tan Heng Hong,5,10,https://twitter.com/MiniMeInsights/status/1244...,False,0,0,0,,pepsi,,,,,,,"[{'user_id': '738937411963490304', 'username':...",,,,,,"[new, colour, change, celebrate, summer, http,..."
1,1244775770842595329,1244772278518386689,1585612645000,2020-03-30 23:57:25,UTC,,"Ever watched the film ""The Invention of Lying""...",[],[],845753663243124736,845753663243124736,TheSolarGazer,Baurice,4,04,https://twitter.com/TheSolarGazer/status/12447...,False,1,0,0,,pepsi,,,,,,,"[{'user_id': '845753663243124736', 'username':...",,,,,,"[ever, watch, film, invention, lie, brilliant,..."
2,1244775765100638213,1244769271663808519,1585612643000,2020-03-30 23:57:23,UTC,,You can try using crystal Pepsi as hand saniti...,[],[],59522423,59522423,BeastieBoy70,Travis Schneden,4,04,https://twitter.com/BeastieBoy70/status/124477...,False,1,1,0,,pepsi,,,,,,,"[{'user_id': '59522423', 'username': 'BeastieB...",,,,,,"[try, use, crystal, hand, sanitizer]"
3,1244775758171639808,1244773330558869505,1585612642000,2020-03-30 23:57:22,UTC,,That is so me. I absolutely hate...no despise ...,[],[],2347992165,2347992165,Keith_Ziegler67,Keith Ziegler,4,04,https://twitter.com/Keith_Ziegler67/status/124...,False,1,1,0,,pepsi,,,,,,,"[{'user_id': '2347992165', 'username': 'Keith_...",,,,,,"[absolutely, hate, despise]"
4,1244775751997624326,1244767774469238784,1585612640000,2020-03-30 23:57:20,UTC,,I Pepsi what you did there.,[],[],67773931,67773931,Thedoctorand,Tim,4,03,https://twitter.com/Thedoctorand/status/124477...,False,0,0,0,,pepsi,,,,,,,"[{'user_id': '67773931', 'username': 'Thedocto...",,,,,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1242581355680202758,1242581355680202758,1585089455000,2020-03-24 22:37:35,UTC,,3 years ago I quit Coke/Pepsi cold turkey. To ...,[],[],8879862,8879862,mvs202,Michael Schade 🚲💨,3,18,https://twitter.com/mvs202/status/124258135568...,False,6,1,0,,pepsi,,,,,,,"[{'user_id': '8879862', 'username': 'mvs202'}]",,,,,☕,"[year, ago, quit, cold, turkey, help, adjust, ..."
156,1242581275107565568,1242581275107565568,1585089436000,2020-03-24 22:37:16,UTC,,"We live in desperate times! Pepsi, really! pi...",[],[],4289106281,4289106281,EntraCod,vkapoor,3,13,https://twitter.com/EntraCod/status/1242581275...,False,3,0,0,,pepsi,,,,,,,"[{'user_id': '4289106281', 'username': 'EntraC...",,,,,,"[live, desperate, time, really]"
157,1242581013563310080,1242581013563310080,1585089374000,2020-03-24 22:36:14,UTC,,"Hey @PapaJohns, we normally have really great ...",[],[],16889710,16889710,LongwoodGeek,Gerry Martin,2,19,https://twitter.com/LongwoodGeek/status/124258...,False,1,2,1,,pepsi,,,,,,,"[{'user_id': '16889710', 'username': 'Longwood...",,,,,,"[hey, papajohns, normally, really, great, serv..."
158,1242580797175013380,1242580797175013380,1585089322000,2020-03-24 22:35:22,UTC,,2 people at Tesco today came in and filled bot...,[],[],702783683607056384,702783683607056384,SydneyJorgs,Sydney,2,05,https://twitter.com/SydneyJorgs/status/1242580...,False,1,0,0,,pepsi,,,,,,,"[{'user_id': '702783683607056384', 'username':...",,,,,🙃,"[people, test, today, come, fill, entire, bask..."


#Exportation en JSON

In [274]:
data = pd.read_csv('df_with_sentiment.csv')
data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns', inplace=True)

data.head(200)  

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_emojis,tweet_cleaned,sentiment
0,1244776216537608193,1244776216537608193,1585612751000,2020-03-30 23:59:11,UTC,NaN,New cold-activated colour changing Pepsi can t...,[],[],738937411963490304,738937411963490304,MiniMeInsights,Tan Heng Hong,5,10,https://twitter.com/MiniMeInsights/status/1244...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '738937411963490304', 'username':...",NaN,NaN,NaN,NaN,NaN,"['new', 'colour', 'change', 'celebrate', 'summ...",1
1,1244775770842595329,1244772278518386689,1585612645000,2020-03-30 23:57:25,UTC,NaN,"Ever watched the film ""The Invention of Lying""...",[],[],845753663243124736,845753663243124736,TheSolarGazer,Baurice,4,4,https://twitter.com/TheSolarGazer/status/12447...,False,1,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '845753663243124736', 'username':...",NaN,NaN,NaN,NaN,NaN,"['ever', 'watch', 'film', 'invention', 'lie', ...",0
2,1244775765100638213,1244769271663808519,1585612643000,2020-03-30 23:57:23,UTC,NaN,You can try using crystal Pepsi as hand saniti...,[],[],59522423,59522423,BeastieBoy70,Travis Schneden,4,4,https://twitter.com/BeastieBoy70/status/124477...,False,1,1,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '59522423', 'username': 'BeastieB...",NaN,NaN,NaN,NaN,NaN,"['try', 'use', 'crystal', 'hand', 'sanitizer']",1
3,1244775758171639808,1244773330558869505,1585612642000,2020-03-30 23:57:22,UTC,NaN,That is so me. I absolutely hate...no despise ...,[],[],2347992165,2347992165,Keith_Ziegler67,Keith Ziegler,4,4,https://twitter.com/Keith_Ziegler67/status/124...,False,1,1,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2347992165', 'username': 'Keith_...",NaN,NaN,NaN,NaN,NaN,"['absolutely', 'hate', 'despise']",1
4,1244775751997624326,1244767774469238784,1585612640000,2020-03-30 23:57:20,UTC,NaN,I Pepsi what you did there.,[],[],67773931,67773931,Thedoctorand,Tim,4,3,https://twitter.com/Thedoctorand/status/124477...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '67773931', 'username': 'Thedocto...",NaN,NaN,NaN,NaN,NaN,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1244408322792587271,1244369936140709888,1585525038000,2020-03-29 23:37:18,UTC,NaN,"When I was a little kid, I had an uncle on the...",[],[],4291294642,4291294642,EWeeks7652,Weeks,5,5,https://twitter.com/EWeeks7652/status/12444083...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '4291294642', 'username': 'EWeeks...",NaN,NaN,NaN,NaN,NaN,"['little', 'kid', 'uncle', 'reservation', 'ove...",1
196,1244408124661997569,1244406576682872840,1585524991000,2020-03-29 23:36:31,UTC,NaN,"""Oh yeah your hair is really adorable, I love ...",[],[],1244231802849542144,1244231802849542144,BabyBremerton,𝑼𝑺𝑺 - 𝑩𝒓𝒆𝒎𝒆𝒓𝒕𝒐𝒏,4,16,https://twitter.com/BabyBremerton/status/12444...,False,0,1,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1244231802849542144', 'username'...",NaN,NaN,NaN,NaN,NaN,"['oh', 'yeah', 'hair', 'really', 'adorable', '...",0
197,1244407938548146177,1244102262407954432,1585524947000,2020-03-29 23:35:47,UTC,NaN,1. Pepsi\n2. Dr.Pepper\n3. Coke\nNo thank you,[],[],761308185537806337,761308185537806337,bshefty9922,Hefty,4,4,https://twitter.com/bshefty9922/status/1244407...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '761308185537806337', 'username':...",NaN,NaN,NaN,NaN,NaN,"['come', 'thank']",0
198,1244407834139295756,1244407834139295756,1585524922000,2020-03-29 23:35:22,UTC,NaN,Got lasagna homemade and pepsi 500ml bottle by...,[],[],849973447602167808,849973447602167808,PeggyEirick,Peggy Eirick,3,21,https://twitter.com/PeggyEirick/status/1244407...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN

## On réctifie les dates

In [273]:
data['date_jour']=datetime.strptime(data['date'][0][:10], ('%Y-%m-%d')).strftime("%A")
data['date']=data['date'][0][:10]
data.head(250)

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_emojis,tweet_cleaned,sentiment,date_jour
0,1244776216537608193,1244776216537608193,1585612751000,2020-03-30,UTC,NaN,New cold-activated colour changing Pepsi can t...,[],[],738937411963490304,738937411963490304,MiniMeInsights,Tan Heng Hong,5,10,https://twitter.com/MiniMeInsights/status/1244...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '738937411963490304', 'username':...",NaN,NaN,NaN,NaN,NaN,"['new', 'colour', 'change', 'celebrate', 'summ...",1,Monday
1,1244775770842595329,1244772278518386689,1585612645000,2020-03-30,UTC,NaN,"Ever watched the film ""The Invention of Lying""...",[],[],845753663243124736,845753663243124736,TheSolarGazer,Baurice,4,4,https://twitter.com/TheSolarGazer/status/12447...,False,1,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '845753663243124736', 'username':...",NaN,NaN,NaN,NaN,NaN,"['ever', 'watch', 'film', 'invention', 'lie', ...",0,Monday
2,1244775765100638213,1244769271663808519,1585612643000,2020-03-30,UTC,NaN,You can try using crystal Pepsi as hand saniti...,[],[],59522423,59522423,BeastieBoy70,Travis Schneden,4,4,https://twitter.com/BeastieBoy70/status/124477...,False,1,1,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '59522423', 'username': 'BeastieB...",NaN,NaN,NaN,NaN,NaN,"['try', 'use', 'crystal', 'hand', 'sanitizer']",1,Monday
3,1244775758171639808,1244773330558869505,1585612642000,2020-03-30,UTC,NaN,That is so me. I absolutely hate...no despise ...,[],[],2347992165,2347992165,Keith_Ziegler67,Keith Ziegler,4,4,https://twitter.com/Keith_Ziegler67/status/124...,False,1,1,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2347992165', 'username': 'Keith_...",NaN,NaN,NaN,NaN,NaN,"['absolutely', 'hate', 'despise']",1,Monday
4,1244775751997624326,1244767774469238784,1585612640000,2020-03-30,UTC,NaN,I Pepsi what you did there.,[],[],67773931,67773931,Thedoctorand,Tim,4,3,https://twitter.com/Thedoctorand/status/124477...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '67773931', 'username': 'Thedocto...",NaN,NaN,NaN,NaN,NaN,[],0,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1244400149440475142,1244400149440475142,1585523090000,2020-03-30,UTC,NaN,Call JG Wentworth 877-CASHNOW!\n@HERMusicx and...,[],[],711670148881514496,711670148881514496,itsreneishab,Run Me My Money,3,16,https://twitter.com/itsreneishab/status/124440...,False,0,0,0,https://twitter.com/Ebony_QT/status/1244391590...,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '711670148881514496', 'username':...",NaN,NaN,NaN,NaN,NaN,"['call', 'j', 'wentworth', 'hermusicx', 'missy...",1,Monday
246,1244400133535608834,1244043107445944321,1585523086000,2020-03-30,UTC,NaN,Mass murderer imbecile Pepsi Trump making It l...,"['#coronavirus', '#covid19', '#pepsitrump', '#...",[],52436759,52436759,keithpp,Keith Parkins,3,15,https://twitter.com/keithpp/status/12444001335...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '52436759', 'username': 'keithpp'...",NaN,NaN,NaN,NaN,NaN,"['mass', 'murderer', 'imbecile', 'tramp', 'mak...",1,Monday
247,1244399936403378176,1244394296998068234,1585523039000,2020-03-30,UTC,NaN,Give Pepsi a little respek,[],[],794562390,794562390,KODaniel7,Kylar O'Daniel,3,2,https://twitter.com/KODaniel7/status/124439993...,False,0,0,0,NaN,pepsi,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '794562390', 'username': 'KODanie...",NaN,NaN,NaN,NaN,NaN,"['give', 'little', 'respect']",1,Monday
248,1244399833261076481,1244399833261076481,1585523014000,2020-03-30,UTC,NaN,Tomorrow is Moday but who cares\nNo waking up ...,[],[],3383022065,3383022065,faitholamini,Uncle Polo,2,19,https://twitter.com/faitholamini/status/1244

## Pour les nuages de mots

In [0]:
import ast

listeH = []
for i in data['hashtags'].values:
  listeH = listeH+ast.literal_eval(i)
listeH = Counter(listeH).most_common(50)
listeH_arranged = []
for i in range(len(listeH)):
  listeH_arranged.append({
    'name': list(listeH[i])[0],
    'weight': list(listeH[i])[1],
  })

listeM = []
for i in data['tweet_cleaned'].values:
  listeM = listeM+ast.literal_eval(i)
listeM = Counter(listeM).most_common(50)
listeM_arranged = []
for i in range(len(listeM)):
  listeM_arranged.append({
    'name': list(listeM[i])[0],
    'weight': list(listeM[i])[1],
  })

listeE = []
for i in data['tweet_emojis'].values:
  listeE.append(i)
listeE = Counter(listeE).most_common(50)
listeE_arranged = []
for i in range(len(listeE)):
  listeE_arranged.append({
    'name': list(listeE[i])[0],
    'weight': list(listeE[i])[1],
  })

In [0]:
Tweets_df.groupby('date_jour').mean().nlikes.values

,created_at,user_id,day,retweet,nlikes,nreplies,nretweets
date_jour,,,,,,,
Monday,1.585553e+12,6.009514e+17,3.882353,False,1.647059,0.294118,0.117647


## On exporte le tout en JSON

In [0]:
data = {}
data['pepsi'] = []
data['pepsi'].append({
    'nbPositif': '',
    'nbNegatif': '',
    'moyenne': '',
    'date': ["01/01/2019","02/01/2019","03/01/2019","04/01/2019","05/01/2019","06/01/2019","07/01/2019"],
    'jour': '',
    'nbLikePos': '',
    'nbRetweetPos': '',
    'nbLikeNeg': '',
    'nbRetweetNeg': '',
    'hashtagAssocie': listeH_arranged,
    'motClefPos': listeM_arranged,
    'motClefNeg': listeM_arranged,
    'emojis': listeE_arranged
})

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [277]:
data

{'pepsi': [{'date': '',
   'emojis': [{'name': nan, 'weight': 3612},
    {'name': '😂', 'weight': 57},
    {'name': '😭', 'weight': 24},
    {'name': '\U0001f97a', 'weight': 15},
    {'name': '😂 😂', 'weight': 14},
    {'name': '😂 😂 😂', 'weight': 14},
    {'name': '👀', 'weight': 13},
    {'name': '🤔', 'weight': 12},
    {'name': '😔', 'weight': 11},
    {'name': '❤', 'weight': 11},
    {'name': '😍', 'weight': 10},
    {'name': '😂 😂 😂 😂', 'weight': 10},
    {'name': '🤣', 'weight': 9},
    {'name': '😩', 'weight': 9},
    {'name': '🙄', 'weight': 8},
    {'name': '💕', 'weight': 8},
    {'name': '😹', 'weight': 8},
    {'name': '😎', 'weight': 8},
    {'name': '😳', 'weight': 8},
    {'name': '😢', 'weight': 8},
    {'name': '😊', 'weight': 7},
    {'name': '😀', 'weight': 7},
    {'name': '\U0001f970', 'weight': 7},
    {'name': '😋', 'weight': 6},
    {'name': '😒', 'weight': 6},
    {'name': '😭 😭 😭', 'weight': 6},
    {'name': '😉', 'weight': 6},
    {'name': '\U0001f92a', 'weight': 6},
    {'name': 

#Idées

##Correction de sentences

In [0]:
from textblob import TextBlob

b = TextBlob("I havv goood speling!")
print(b.correct())

I have good spelling!


##Tokénisation

In [0]:
nltk.pos_tag(['go'])

[('go', 'VB')]

##Emotions *par mot*

In [0]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('love'))

breakdown = swn.senti_synset('kill.v.01')
print("pos_score: " , breakdown.pos_score())
print("neg_score: " , breakdown.neg_score())
print("obj_score: " , breakdown.obj_score())


pos_score:  0.0
neg_score:  0.5
obj_score:  0.5


##2eme essai de cleaning

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer 

#create an object of class PorterStemmer
stop_words = set(stopwords.words('english'))   

for i in range(len(Tweets_df)):
  word_tokens = word_tokenize(Tweets_df['tweet'][i])   
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = []   
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  #print(word_tokens) 
  #print(filtered_sentence)
  tweet_rmStop.append(filtered_sentence)
  
tweet_lemm = tweet_rmStop

In [0]:
# import these modules 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

for i in range(len(tweet_rmStop)):
  for j in range(len(tweet_rmStop[i])):
    tweet_lemm[i][j] = porter.stem(tweet_rmStop[i][j])
    print(tweet_lemm[i][j])

In [0]:
import pandas
#Tweets_df = pandas.DataFrame()

Tweets_df["tweet_rmStop"]= tweet_rmStop
Tweets_df["tweet_lemm"] = tweet_lemm
Tweets_df["tweet_emojis"] = emojisExtracted